In [1]:
import h5py
import json
import numpy as np
import os
import pandas as pd

img_data = './data/img_data'
text_data = './data/text_data'

img_features = np.asarray(h5py.File(os.path.join(img_data, 'IR_image_features.h5'), 'r')['img_features'])

with open(os.path.join(img_data, 'IR_img_features2id.json'), 'r') as f:
     visual_feat_mapping = json.load(f)['IR_imgid2id']

In [2]:
# df = pd.read_json(os.path.join(text_data, 'Easy/IR_train_easy.json'))
# df = df.T.sort_index()
# df.dialog = df.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df['all_words'] = df[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df = df[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df = df.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df['target_onehot'] = np.where(df['target_img_id'] == df['img_id'], 1, 0)
# s = []
# for img_id in df.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df['img_feat'] = pd.Series(s)
# df[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/train_data_easy_bow.csv')

In [3]:
# df_val = pd.read_json(os.path.join(text_data, 'Easy/IR_val_easy.json'))
# df_val = df_val.T.sort_index()
# df_val.dialog = df_val.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df_val['all_words'] = df_val[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df_val = df_val[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df_val.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df_val = df_val.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df_val['target_onehot'] = np.where(df_val['target_img_id'] == df_val['img_id'], 1, 0)
# s = []
# for img_id in df_val.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df_val['img_feat'] = pd.Series(s)
# df_val[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/val_data_easy_bow.csv')

In [4]:
# df_test = pd.read_json(os.path.join(text_data, 'Easy/IR_test_easy.json'))
# df_test = df_test.T.sort_index()
# df_test.dialog = df_test.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df_test['all_words'] = df_test[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df_test = df_test[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df_test.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df_test = df_test.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df_test['target_onehot'] = np.where(df_test['target_img_id'] == df_test['img_id'], 1, 0)
# s = []
# for img_id in df_test.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df_test['img_feat'] = pd.Series(s)
# df_test[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/test_data_easy_bow.csv')

In [31]:
for data in ['train', 'val', 'test']:
    for complexity in ['easy', 'hard']:
        print(data, complexity)
        df = pd.read_json(os.path.join(text_data, '{0}/IR_{1}_{2}.json'\
                                       .format(complexity.capitalize(), data, complexity)))
        df = df.T.sort_index()
        df.dialog = df.dialog\
            .apply(lambda arr: ' \n '.join([sent for item in arr for sent in item]))
        df['all_words'] = df[['caption', 'dialog']]\
            .apply(lambda x: x[0] + ' \n ' + x[1], axis = 1)
        df = df[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
        s = df.apply(lambda x: pd.Series(x['img_list']),axis=1)\
            .stack()\
            .reset_index(level=1, drop=True)
        s.name = 'img_id'
        df = df.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
        df['target_onehot'] = np.where(df['target_img_id'] == df['img_id'], 1, 0)
        
        df[['all_words', 'img_id', 'target_onehot']].to_csv('./data/{0}_data_{1}_bow.csv'\
                                                             .format(data, complexity), 
                                                              index = False)
        

train easy
train hard
val easy
val hard
test easy
test hard


In [36]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

class ImageTextDataSet(Dataset):

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.imagetext_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.imagetext_frame)

    def __getitem__(self, idx):
        text = self.imagetext_frame.iloc[idx, 0].split(' \n ')
        img_id = self.imagetext_frame.iloc[idx, 1]
        target = self.imagetext_frame.iloc[idx, 2]
        h5_id = visual_feat_mapping[str(img_id)]
        img_feat = img_features[h5_id]
        sample = {'text': text, 'img_features': img_feat, 'target': target}
        return sample

In [37]:
data_train = ImageTextDataSet(csv_file='./data/train_data_easy_bow.csv')
dataloader_train = DataLoader(data_train, batch_size=4,
                        shuffle=True, num_workers=4)

data_val = ImageTextDataSet(csv_file='./data/val_data_easy_bow.csv')
dataloader_val = DataLoader(data_val, num_workers=4)

data_test = ImageTextDataSet(csv_file='./data/test_data_easy_bow.csv')
dataloader_test = DataLoader(data_test, num_workers=4)


In [ ]:
# coding: utf-8

"""
CBOW

Based on Graham Neubig's DyNet code examples:
  https://github.com/neubig/nn4nlp2017-code
  http://phontron.com/class/nn4nlp2017/

"""

from collections import defaultdict
import time
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<unk>"]

def read_dataset(dataloader):
    for batch in dataloader:
        text = batch['text']
        for sentences in text:
            for sentence in sentences:
                resulting_sentence = sentence.lower().strip()
                yield ([w2i[x] for x in resulting_sentence.split(" ")])


# Read in the data
train = list(read_dataset(dataloader_train))
w2i = defaultdict(lambda: UNK, w2i)
val = list(read_dataset(dataloader_val))
test = list(read_dataset(dataloader_test))

nwords = len(w2i)

In [ ]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, output_dim)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        bow = torch.sum(embeds, 1)
        logits = self.linear(bow)
        return logits


model = CBOW(nwords, 64, ntags)
print(model)


def evaluate(model, data):
    """Evaluate a model on a data set."""
    correct = 0.0
    
    for words, tag in data:
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        predict = scores.data.numpy().argmax(axis=1)[0]

        if predict == tag:
            correct += 1

    return correct, len(data), correct/len(data)


optimizer = optim.SGD(model.parameters(), lr=0.01)

for ITER in range(100):

    random.shuffle(train)
    train_loss = 0.0
    start = time.time()

    for words, tag in train:

        # forward pass
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        loss = nn.CrossEntropyLoss()
        target = Variable(torch.LongTensor([tag]))
        output = loss(scores, target)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: train loss/sent=%.4f, time=%.2fs" % 
          (ITER, train_loss/len(train), time.time()-start))

    # evaluate
    _, _, acc = evaluate(model, dev)
    print("iter %r: test acc=%.4f" % (ITER, acc))
